## 데이터 획득 준비작업

In [120]:
import pandas as pd
import numpy as np

import platform
import matplotlib.pyplot as plt

%matplotlib inline 

path = "c:/Windows/Fonts/malgun.ttf"
from matplotlib import font_manager, rc
if platform.system() == 'Darwin':
    rc('font', family='AppleGothic')
elif platform.system() == 'Windows':
    font_name = font_manager.FontProperties(fname=path).get_name()
    rc('font', family=font_name)
else:
    print('Unknown system... sorry~~~~')    

plt.rcParams['axes.unicode_minus'] = False

In [121]:
from selenium import webdriver
import time

#### 개정된 변경내용으로 아래와 같이 작성해야 함

In [122]:
driver = webdriver.Chrome('../driver/chromedriver')
driver.get("http://info.nec.go.kr/main/showDocument.xhtml?electionId=0000000000&topMenuId=VC&secondMenuId=VCCP09")

#### 대통령선거라는 글자부분을 클릭합니다. 해당 글자에서 앞 장에서 설명한 크롬 개발자 도구로 코드를 확인하면 id가 electionType1으로 나타납니다.

In [123]:
driver.find_element_by_id("electionType1").click() 

#### 그리고 나타나는 선택항목에서 제19대를 선택하도록 합니다. 역시 크롬 개발자 도구에서 해당 위치의 id는 electionName이라는 것을 찾을 수 있습니다.

In [124]:
driver.find_element_by_id("electionName").send_keys("제19대") 

#### 그리고 나타나는 선택에서 대통령선거를 선택하도록 합니다.

In [125]:
driver.find_element_by_id("electionCode").send_keys("대통령선거") 

#### 이제 나타나는 시도 항목에서 선택 부분의 XPath를 찾고, 해당 리스트를 가져옵니다

In [126]:
sido_list_raw = driver.find_element_by_xpath("""//*[@id="cityCode"]""")
sido_list = sido_list_raw.find_elements_by_tag_name("option")
sido_names_values = [option.text for option in sido_list]
sido_names_values = sido_names_values[2:]
sido_names_values

['서울특별시',
 '부산광역시',
 '대구광역시',
 '인천광역시',
 '광주광역시',
 '대전광역시',
 '울산광역시',
 '세종특별자치시',
 '경기도',
 '강원도',
 '충청북도',
 '충청남도',
 '전라북도',
 '전라남도',
 '경상북도',
 '경상남도',
 '제주특별자치도']

## 6-2. 19대 대선 개표 결과 데이터 획득하기

<img src="../img/002.jpg" width="600" height="400"></img>

* 위 그림이 지역을 선택하면 나타나는 화면중 하나인데 득표수에 득표율이 괄호()로 함께 나타납니다.
* 이를 제거하고 **(**를 기준으로 왼쪽 숫자만 얻어서, 콤마(,)를 제거하고, float형으로 변경하는 함수를 **get_num**으로 준비해 둡니다

In [152]:
import re

def get_num(temp):
    return float(re.split('\(',temp)[0].replace(',',''))

* 책에서는 특별히 다루고 있지 않지만, 아래 모듈을 import해서 wait.until 함수를 사용할 수 있습니다.

<img src="../img/003.jpg" width="600" height="200"></img>

* 이 함수는 **검색**버튼이 클릭가능할 때 까지 기다리는 기능을 합니다.
* 그리고 **move_sido** 함수는 광역시도 이름을 리스트에 전송하고 검색 버튼을 누르는 역할을 합니다.

In [154]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

wait = WebDriverWait(driver,10)

def move_sido(name):
    element = driver.find_element_by_id("cityCode")
    element.send_keys(name)
    make_xpath = """//*[@id="searchBtn"]"""
    wait.until(EC.element_to_be_clickable((By.XPATH,make_xpath)))
    driver.find_element_by_xpath(make_xpath).click() 

***append_data** 함수는 빈 내용으로 미리 준비한 DataFrame에 append 명령으로 읽은 데이터를 하나씩 추가하는 기능입니다.

<img src="../img/002.jpg" width="600" height="400"></img>

* 이 함수가 실행되면 전체 투표수, 문재인후보, 홍준표후보, 안철수후보의 득표수가 추가됩니다

In [155]:
def append_data(df, sido_name, data):
    for each in df[0].values[1:]:
        data['광역시도'].append(sido_name)
        data['시군'].append(each[0])
        data['pop'].append(get_num(each[2]))
        data['moon'].append(get_num(each[3]))
        data['hong'].append(get_num(each[4]))
        data['ahn'].append(get_num(each[5]))

미리 변수를 하나 만들어 둡니다.

In [156]:
election_result_raw = {'광역시도' : [],
                       '시군' : [],
                       'pop' : [],
                       'moon' : [],
                       'hong' : [],    
                       'ahn' : [] }

#### 이후 에러, 이후 검토 못하고 올립니다. 뵙고.. 정리하겠습니다.

In [157]:
from bs4 import BeautifulSoup

for each_sido in sido_names_values:
    move_sido(each_sido)
    
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    table = soup.find('table')
    
    df = pd.read_html(str(table))
    
    append_data(df, sido_name, election_result_raw)

TypeError: expected string or bytes-like object

In [ ]:
election_result = pd.DataFrame(election_result_raw, 
                               columns=['광역시도', '시군', 'pop', 'moon','hong','ahn'])
election_result

In [ ]:
election_result.to_csv('../data/05. election_result.csv', encoding='utf-8', sep=',')

In [ ]:
driver.close()